In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

In [37]:
def habr_search(*queries, num_pages=1):
    url = 'https://habr.com/ru/search/'
    all_results = []
    
    for query in queries:
        for page in range(1, num_pages + 1):
            params = {'q': query, 'page': page}
            response = requests.get(url, params=params)
            time.sleep(0.2)
            soup = BeautifulSoup(response.text, 'html.parser')
            articles = soup.find_all('article', class_ = 'tm-articles-list__item')
        
            for article in articles:
                # Извлекаем дату
                time_tag = article.find('time')
                if time_tag:
                    datetime = time_tag['title']
                else:
                    datetime = 'Нет даты'
                    
                # Извлекаем заголок    
                title_tag = article.find('a', class_ = 'tm-title__link')
            
                if title_tag:
                    title = title_tag.find('span').get_text()
                    
                    # Извлекаем полную ссылку на статью
                    link = 'https://habr.com' + title_tag['href']
                else:
                    continue  # Пропускаем статью, если заголовок не найден
                
                
                article_response = requests.get(link)
                article_response.encoding = 'utf-8'
                article_soup = BeautifulSoup(article_response.text, 'html.parser')
                
                # Извлекаем полныц текст статьи
                content_tag = article_soup.find('div', class_='article-formatted-body')
                if content_tag:
                    article_text = content_tag.get_text(separator=' ').replace('\n', ' ').strip()
                else:
                    article_text = 'Текст недоступен'
                    
                # Извлекаем количество лайков
                like_tag = article_soup.find('span', class_='tm-votes-meter__value')
                if like_tag:
                    likes = like_tag.get_text()
                else:
                    likes = 'Нет лайков'
                    
                # Добавляем результат в список
                all_results.append((datetime, title, link, article_text, likes))
    
    
    unique_results = list(set(all_results))
    
    # Преобразуем в DataFrame
    df = pd.DataFrame(unique_results, columns=['Дата', 'Заголовок', 'Ссылка', 'Текст материала', 'Количество лайков'])
    
    return df

In [33]:
df_habr = habr_search('python', 'анализ данных', num_pages=2)

In [35]:
df_habr = df_habr.sort_values(by='Дата', ascending=False)

In [36]:
df_habr

,Дата,Заголовок,Ссылка,Текст материала,Количество лайков
26,"2024-09-17, 16:00",ИТМО провёл исследование open source в сферах ...,https://habr.com/ru/news/844072/,Специалисты центра «Сильный ИИ в промышленност...,Нет лайков
17,"2024-09-02, 10:27",55% Python-разработчиков используют Linux-окру...,https://habr.com/ru/news/840224/,Организация Python Software Foundation и компа...,+10
10,"2024-08-14, 16:20",IBM анонсировала бесплатные курсы в области ан...,https://habr.com/ru/news/836040/,IBM - одна из ведущих компаний в области инфор...,+1
36,"2024-07-11, 09:57",Инсайдерам Microsoft 365 станет доступен редак...,https://habr.com/ru/news/828098/,В сентябре 2023 года Microsoft запустила над...,+8
29,"2024-07-01, 14:46",Релиз открытой библиотеки для быстрой обработк...,https://habr.com/ru/news/825868/,1 июля 2024 года состоялся релиз открытой би...,+3
1,"2024-05-09, 07:43",Вышла первая бета-версия языка программировани...,https://habr.com/ru/news/813271/,8 мая 2024 года вышла первая бета-версия язы...,+8
20,"2024-04-27, 10:20",Начните учиться бесплатно на курсе «Python для...,https://habr.com/ru/companies/yandex_praktikum...,Мы добавили бесплатный вводный модуль к курсу ...,+4
0,"2023-11-09, 14:05",Alchemer совместно с JetBrains и Python Softwa...,https://habr.com/ru/news/772872/,Alchemer совместно с JetBrains и Python Softwa...,+3
33,"2023-10-20, 07:17",Автор курсов по Python и Pandas жалуется на ве...,https://habr.com/ru/news/768702/,DALL-E 3 \n \nПреподаватель языка Python Ревер...,+13
30,"2023-10-09, 15:23",Три уровня погружения в Python. Запись докладо...,https://habr.com/ru/companies/selectel/news/76...,"Привет, Хабр! В сентябре мы провели Selectel P...",+2
